In [1]:
from langchain import hub
# obj = ChatPromptTemplate()
obj = hub.pull("hardkothari/prompt-maker")

In [4]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()
# prompt = llm(obj.format_messages(task="search and summarize artificial intelligence research paper", lazy_prompt="you are a expert about searching and summarize about research paper"))
prompt = llm(obj.format_messages(task="Extract the name of a research topic or research paper", lazy_prompt="Extract a research topic or research paper name from a given user input. Do not return any words other than the topic or name"))

print(prompt.content)

"""
As an expert in researching and summarizing research papers, your task is to conduct a comprehensive search on a given topic and provide a concise summary of the key findings and insights from the selected papers. Your summary should include an overview of the research question, methodology, results, and conclusions of each paper. Additionally, please organize the summaries in a logical and coherent manner, highlighting any common themes or patterns that emerge from the research. Aim to provide a well-structured and informative summary that can serve as a valuable resource for readers seeking a quick understanding of the current state of research on the topic.
"""


As an expert in research topics and papers, your task is to develop an algorithm that can accurately extract the name of a research topic or research paper from a given user input. The algorithm should only return the topic or name and exclude any other words. 

### Context:
You are working on a natural language processing project and have been provided with a dataset of user inputs containing various research topics and paper titles. These inputs may include additional words or phrases that are not part of the actual topic or name.

### Instructions:
Develop an algorithm that can identify and extract the research topic or paper name from a given user input. The algorithm should consider different variations of input formats, such as sentences, phrases, or even single words. It should be able to handle different languages and take into account common research-related words or phrases.

### Outcome:
Your algorithm should return the extracted research topic or paper name accurately, excl

'\nAs an expert in researching and summarizing research papers, your task is to conduct a comprehensive search on a given topic and provide a concise summary of the key findings and insights from the selected papers. Your summary should include an overview of the research question, methodology, results, and conclusions of each paper. Additionally, please organize the summaries in a logical and coherent manner, highlighting any common themes or patterns that emerge from the research. Aim to provide a well-structured and informative summary that can serve as a valuable resource for readers seeking a quick understanding of the current state of research on the topic.\n'

In [13]:
import os

from flask import Flask, Response, request, render_template
import threading
import queue

from langchain.chat_models import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.schema import AIMessage, HumanMessage, SystemMessage

from metaphor_python import Metaphor
from langchain.agents import tool
from typing import List

from langchain.schema import StrOutputParser
from langchain.prompts import ChatPromptTemplate

class ChainStreamHandler(StreamingStdOutCallbackHandler):
    def __init__(self, gen):
        super().__init__()
        self.gen = gen

    def on_llm_new_token(self, token: str, **kwargs):
        self.gen.send(token)
        
class ThreadedGenerator:
    def __init__(self):
        self.queue = queue.Queue()

    def __iter__(self):
        return self

    def __next__(self):
        item = self.queue.get()
        if item is StopIteration: raise item
        return item

    def send(self, data):
        self.queue.put(data)

    def close(self):
        self.queue.put(StopIteration)

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", verbose=True, streaming=True, callbacks=[ChainStreamHandler(ThreadedGenerator())], temperature=0.7)
prompt = ChatPromptTemplate.from_messages([
    ("system", "As an expert in researching and summarizing research papers, your task is to conduct a comprehensive search on a given topic and provide a concise summary of the key findings and insights from the selected papers. Your summary should include an overview of the research question, methodology, results, and conclusions of each paper. Additionally, please organize the summaries in a logical and coherent manner, highlighting any common themes or patterns that emerge from the research. Aim to provide a well-structured and informative summary that can serve as a valuable resource for readers seeking a quick understanding of the current state of research on the topic."),
    ("human", "{question}"),    
])
chain = prompt | llm | StrOutputParser()


chain.invoke({"question" : "Tell me a joke"})